# Problem Statement

### Credit Card Lead Prediction

Happy Customer Bank is a mid-sized private bank that deals in all kinds of banking products, like Savings accounts, Current accounts, investment products, credit products, among other offerings.


The bank also cross-sells products to its existing customers and to do so they use different kinds of communication like tele-calling, e-mails, recommendations on net banking, mobile banking, etc. 


In this case, the Happy Customer Bank wants to cross sell its credit cards to its existing customers. The bank has identified a set of customers that are eligible for taking these credit cards.


Now, the bank is looking for your help in identifying customers that could show higher intent towards a recommended credit card, given:

    Customer details (gender, age, region etc.)
    Details of his/her relationship with the bank (Channel_Code,Vintage, 'Avg_Asset_Value etc.)


In [56]:
# !wget https://datahack-prod.s3.amazonaws.com/train_file/train_s3TEQDk.csv
# !wget https://datahack-prod.s3.amazonaws.com/test_file/test_mSzZ8RL.csv
# pip install -U scikit-learn

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

pd.options.display.max_columns = 300

In [58]:
data = pd.read_csv("train_s3TEQDk.csv")
test = pd.read_csv("test_mSzZ8RL.csv")
print(f"Train shape {data.shape}, Test Shape {test.shape}")

Train shape (245725, 11), Test Shape (105312, 10)


In [59]:
train,valid = train_test_split(data,test_size=0.20,random_state=345,stratify=data['Is_Lead'])
train = train.copy()
valid = valid.copy()
print(f"Train shape {train.shape} Validation shape {valid.shape}")

Train shape (196580, 11) Validation shape (49145, 11)


In [60]:
train.head(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
53078,N3ZQ84QR,Female,46,RG280,Self_Employed,X2,51,No,863584,Yes,0
213644,JWGAMK7P,Male,67,RG258,Other,X2,43,Yes,706126,No,0
131870,CX9NGNQT,Male,46,RG279,Self_Employed,X2,26,Yes,422207,Yes,0


In [61]:
test.head(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,VBENBARO,Male,29,RG254,Other,X1,25,Yes,742366,No
1,CCMEWNKY,Male,43,RG268,Other,X2,49,NaN,925537,No
2,VK3KGA9M,Male,31,RG270,Salaried,X1,14,No,215949,No


In [62]:
valid.head(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
148453,OK9KJGZ2,Female,55,RG268,Self_Employed,X1,37,No,929257,No,0
117997,TTC7CPSI,Male,57,RG283,Self_Employed,X3,87,NaN,909740,No,0
5432,MPUWVRAX,Male,39,RG275,Salaried,X1,8,Yes,961742,Yes,0


In [63]:
train['ID'].nunique()

196580

In [64]:
train.isna().sum()

ID                         0
Gender                     0
Age                        0
Region_Code                0
Occupation                 0
Channel_Code               0
Vintage                    0
Credit_Product         23525
Avg_Account_Balance        0
Is_Active                  0
Is_Lead                    0
dtype: int64

In [65]:
train['Gender'].value_counts(normalize=True)

Male      0.546693
Female    0.453307
Name: Gender, dtype: float64

In [66]:
train['Region_Code'].nunique()

35

In [67]:
train['Occupation'].value_counts(normalize=True)

Self_Employed    0.411532
Salaried         0.292731
Other            0.284866
Entrepreneur     0.010871
Name: Occupation, dtype: float64

In [68]:
train['Channel_Code'].value_counts(normalize=True)

X1    0.421279
X3    0.280359
X2    0.275669
X4    0.022693
Name: Channel_Code, dtype: float64

In [69]:
train['Credit_Product'].value_counts(normalize=True)

No     0.667019
Yes    0.332981
Name: Credit_Product, dtype: float64

In [70]:
train['Avg_Account_Balance'].describe()

count    1.965800e+05
mean     1.129489e+06
std      8.532486e+05
min      2.079000e+04
25%      6.042470e+05
50%      8.954865e+05
75%      1.368733e+06
max      1.035201e+07
Name: Avg_Account_Balance, dtype: float64

In [71]:
train['Is_Active'].value_counts(normalize=True)

No     0.611375
Yes    0.388625
Name: Is_Active, dtype: float64

In [72]:
train['Is_Lead'].value_counts(normalize=True)

0    0.762794
1    0.237206
Name: Is_Lead, dtype: float64

In [73]:
train['Age'].describe()

count    196580.000000
mean         43.864971
std          14.821238
min          23.000000
25%          30.000000
50%          43.000000
75%          54.000000
max          85.000000
Name: Age, dtype: float64

In [74]:
train['Vintage'].describe()

count    196580.000000
mean         46.978121
std          32.346981
min           7.000000
25%          20.000000
50%          32.000000
75%          73.000000
max         135.000000
Name: Vintage, dtype: float64

In [75]:
train.groupby(['Is_Lead'])[['Age','Avg_Account_Balance']].median()

,Age,Avg_Account_Balance
Is_Lead,,
0,38,871158
1,49,980686


In [76]:
train['Credit_Product'] = train['Credit_Product'].fillna('NA')
train['Avg_Account_Balance'] = np.log(1+train['Avg_Account_Balance'])

test['Credit_Product'] = test['Credit_Product'].fillna('NA')
test['Avg_Account_Balance'] = np.log(1+test['Avg_Account_Balance'])

valid['Credit_Product'] = valid['Credit_Product'].fillna('NA')
valid['Avg_Account_Balance'] = np.log(1+valid['Avg_Account_Balance'])


In [77]:
train.sample(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
34,FXPTJYP7,Male,67,RG268,Other,X1,87,Yes,14.336791,No,1
196199,GQLACHBW,Male,49,RG283,Other,X3,98,NA,13.735415,No,1
142727,MDKVS4YD,Female,42,RG275,Self_Employed,X1,32,No,13.775077,No,0


In [78]:
train['Is_Active'] = train['Is_Active'].replace({'No':'N','Yes':'Y'})
test['Is_Active'] = test['Is_Active'].replace({'No':'N','Yes':'Y'})
valid['Is_Active'] = valid['Is_Active'].replace({'No':'N','Yes':'Y'})

In [79]:
def bucket_age(age):
    if age <= 25:
        val = '0-25'
    elif age > 25 and age <=30:
        val = '25-30'
    elif age > 30 and age <= 35:
        val = '30-35'
    elif age > 35 and age <=40:
        val = '35-40'
    elif age > 40 and age <=45:
        val = '40-45'
    elif age > 45 and age <=50:
        val = '45-50'
    elif age > 50 and age <= 55:
        val = '50-55'
    elif age > 55 and age <= 60:
        val = '55-60'
    elif age > 60 and age <=65:
        val = '60-65'
    elif age >65 and age <=70:
        val = '65-70'
    else:
        val = '70+'
    return val

train['age_cat'] = train['Age'].apply(bucket_age)
test['age_cat'] = test['Age'].apply(bucket_age)
valid['age_cat'] = valid['Age'].apply(bucket_age)
train[['Age','age_cat']].sample(3)

,Age,age_cat
158063,32,30-35
29994,24,0-25
185735,66,65-70


In [80]:
train[['Age','age_cat']].sample(3)

,Age,age_cat
241090,60,55-60
62338,33,30-35
126424,39,35-40


In [81]:
cat_cols = ['age_cat','Gender','Region_Code','Occupation','Channel_Code','Credit_Product','Is_Active']
featured_cols = []
for idx,col in enumerate(cat_cols):
    for sub_col in cat_cols[idx+1:]:
        new_col = f"{col}-{sub_col}"
        featured_cols.append(new_col)
        train[new_col] = train[col] + "-" + train[sub_col]
        test[new_col] = test[col] + "-" + test[sub_col]
        valid[new_col] = valid[col] + "-" + valid[sub_col]

train.sample(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead,age_cat,age_cat-Gender,age_cat-Region_Code,age_cat-Occupation,age_cat-Channel_Code,age_cat-Credit_Product,age_cat-Is_Active,Gender-Region_Code,Gender-Occupation,Gender-Channel_Code,Gender-Credit_Product,Gender-Is_Active,Region_Code-Occupation,Region_Code-Channel_Code,Region_Code-Credit_Product,Region_Code-Is_Active,Occupation-Channel_Code,Occupation-Credit_Product,Occupation-Is_Active,Channel_Code-Credit_Product,Channel_Code-Is_Active,Credit_Product-Is_Active
78094,DZ9HTUFA,Male,52,RG277,Other,X2,67,Yes,13.832029,N,0,50-55,50-55-Male,50-55-RG277,50-55-Other,50-55-X2,50-55-Yes,50-55-N,Male-RG277,Male-Other,Male-X2,Male-Yes,Male-N,RG277-Other,RG277-X2,RG277-Yes,RG277-N,Other-X2,Other-Yes,Other-N,X2-Yes,X2-N,Yes-N
60648,MNMJGZ2J,Male,50,RG281,Self_Employed,X3,74,Yes,13.201117,N,0,45-50,45-50-Male,45-50-RG281,45-50-Self_Employed,45-50-X3,45-50-Yes,45-50-N,Male-RG281,Male-Self_Employed,Male-X3,Male-Yes,Male-N,RG281-Self_Employed,RG281-X3,RG281-Yes,RG281-N,Self_Employed-X3,Self_Employed-Yes,Self_Employed-N,X3-Yes,X3-N,Yes-N
204946,HF25RAVK,Female,42,RG257,Other,X2,57,Yes,13.162801,Y,0,40-45,40-45-Female,40-45-RG257,40-45-Other,40-45-X2,40-45-Yes,40-45-Y,Female-RG257,Female-Other,Female-X2,Female-Yes,Female-Y,RG257-Other,RG257-X2,RG257-Yes,RG257-Y,Other-X2,Other-Yes,Other-Y,X2-Yes,X2-Y,Yes-Y


In [82]:
all_cat_cols = cat_cols + featured_cols
num_col = ['Vintage','Avg_Account_Balance','Is_Lead']
for idx,col in enumerate (all_cat_cols):
    print(f"\rWorking Cat Col {idx}/{len(all_cat_cols)}",end='')
    for ind, num in enumerate(num_col):

        grp = train.groupby([col])[num].agg(['mean','std'])
        grp['cv'] = grp['std']/(1+grp['mean'])
        grp = grp.add_prefix(f'{col}-{num}-')
        grp = grp.fillna(-1)
        grp = grp.reset_index()
        
        train = train.merge(grp,on=[col],how='left')
        test = test.merge(grp,on=[col],how='left')
        valid = valid.merge(grp,on=[col],how='left')

Working Cat Col 27/28

In [83]:
train.head(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead,age_cat,age_cat-Gender,age_cat-Region_Code,age_cat-Occupation,age_cat-Channel_Code,age_cat-Credit_Product,age_cat-Is_Active,Gender-Region_Code,Gender-Occupation,Gender-Channel_Code,Gender-Credit_Product,Gender-Is_Active,Region_Code-Occupation,Region_Code-Channel_Code,Region_Code-Credit_Product,Region_Code-Is_Active,Occupation-Channel_Code,Occupation-Credit_Product,Occupation-Is_Active,Channel_Code-Credit_Product,Channel_Code-Is_Active,Credit_Product-Is_Active,age_cat-Vintage-mean,age_cat-Vintage-std,age_cat-Vintage-cv,age_cat-Avg_Account_Balance-mean,age_cat-Avg_Account_Balance-std,age_cat-Avg_Account_Balance-cv,age_cat-Is_Lead-mean,age_cat-Is_Lead-std,age_cat-Is_Lead-cv,Gender-Vintage-mean,Gender-Vintage-std,Gender-Vintage-cv,Gender-Avg_Account_Balance-mean,Gender-Avg_Account_Balance-std,Gender-Avg_Account_Balance-cv,Gender-Is_Lead-mean,Gender-Is_Lead-std,Gender-Is_Lead-cv,Region_Code-Vintage-mean,Region_Code-Vintage-std,Region_Code-Vintage-cv,Region_Code-Avg_Account_Balance-mean,Region_Code-Avg_Account_Balance-std,Region_Code-Avg_Account_Balance-cv,Region_Code-Is_Lead-mean,Region_Code-Is_Lead-std,Region_Code-Is_Lead-cv,Occupation-Vintage-mean,Occupation-Vintage-std,Occupation-Vintage-cv,Occupation-Avg_Account_Balance-mean,Occupation-Avg_Account_Balance-std,Occupation-Avg_Account_Balance-cv,Occupation-Is_Lead-mean,Occupation-Is_Lead-std,Occupation-Is_Lead-cv,Channel_Code-Vintage-mean,Channel_Code-Vintage-std,Channel_Code-Vintage-cv,Channel_Code-Avg_Account_Balance-mean,Channel_Code-Avg_Account_Balance-std,Channel_Code-Avg_Account_Balance-cv,Channel_Code-Is_Lead-mean,Channel_Code-Is_Lead-std,Channel_Code-Is_Lead-cv,Credit_Product-Vintage-mean,Credit_Product-Vintage-std,Credit_Product-Vintage-cv,Credit_Product-Avg_Account_Balance-mean,Credit_Product-Avg_Account_Balance-std,Credit_Product-Avg_Account_Balance-cv,Credit_Product-Is_Lead-mean,Credit_Product-Is_Lead-std,Credit_Product-Is_Lead-cv,Is_Active-Vintage-mean,Is_Active-Vintage-std,Is_Active-Vintage-cv,Is_Active-Avg_Account_Balance-mean,Is_Active-Avg_Account_Balance-std,Is_Active-Avg_Account_Balance-cv,Is_Active-Is_Lead-mean,Is_Active-Is_Lead-std,Is_Active-Is_Lead-cv,age_cat-Gender-Vintage-mean,age_cat-Gender-Vintage-std,age_cat-Gender-Vintage-cv,age_cat-Gender-Avg_Account_Balance-mean,age_cat-Gender-Avg_Account_Balance-std,age_cat-Gender-Avg_Account_Balance-cv,age_cat-Gender-Is_Lead-mean,age_cat-Gender-Is_Lead-std,age_cat-Gender-Is_Lead-cv,age_cat-Region_Code-Vintage-mean,age_cat-Region_Code-Vintage-std,age_cat-Region_Code-Vintage-cv,age_cat-Region_Code-Avg_Account_Balance-mean,age_cat-Region_Code-Avg_Account_Balance-std,age_cat-Region_Code-Avg_Account_Balance-cv,age_cat-Region_Code-Is_Lead-mean,age_cat-Region_Code-Is_Lead-std,age_cat-Region_Code-Is_Lead-cv,age_cat-Occupation-Vintage-mean,age_cat-Occupation-Vintage-std,age_cat-Occupation-Vintage-cv,age_cat-Occupation-Avg_Account_Balance-mean,age_cat-Occupation-Avg_Account_Balance-std,age_cat-Occupation-Avg_Account_Balance-cv,age_cat-Occupation-Is_Lead-mean,age_cat-Occupation-Is_Lead-std,age_cat-Occupation-Is_Lead-cv,age_cat-Channel_Code-Vintage-mean,age_cat-Channel_Code-Vintage-std,age_cat-Channel_Code-Vintage-cv,age_cat-Channel_Code-Avg_Account_Balance-mean,age_cat-Channel_Code-Avg_Account_Balance-std,age_cat-Channel_Code-Avg_Account_Balance-cv,age_cat-Channel_Code-Is_Lead-mean,age_cat-Channel_Code-Is_Lead-std,age_cat-Channel_Code-Is_Lead-cv,age_cat-Credit_Product-Vintage-mean,age_cat-Credit_Product-Vintage-std,age_cat-Credit_Product-Vintage-cv,age_cat-Credit_Product-Avg_Account_Balance-mean,age_cat-Credit_Product-Avg_Account_Balance-std,age_cat-Credit_Product-Avg_Account_Balance-cv,age_cat-Credit_Product-Is_Lead-mean,age_cat-Credit_Product-Is_Lead-std,age_cat-Credit_Product-Is_Lead-cv,age_cat-Is_Active-Vintage-mean,age_cat-Is_Active-Vintage-std,age_cat-Is_Active-Vintage-cv,age_cat-Is_Active-Avg_Account_Balance-me

In [84]:
all_cat_cols = cat_cols + featured_cols
encoder = LabelEncoder()
for col in all_cat_cols:
    train[col] = encoder.fit_transform(train[col])
    test[col] = encoder.transform(test[col])
    valid[col] = encoder.transform(valid[col])

In [85]:
x_train = train.drop(['ID','Is_Lead'],axis=1)
y_train = train['Is_Lead']

x_valid = valid.drop(['ID','Is_Lead'],axis=1)
y_valid = valid['Is_Lead']

x_test = test.drop(['ID'],axis=1)

In [86]:
cat_indx = []
for idx, col in enumerate(x_train.columns):
    if col in all_cat_cols:
        cat_indx.append(idx)

In [89]:
hist_params = {'max_iter':1000,
               'learning_rate' : 0.05,
               'max_depth' : 30,
               'early_stopping' : 'auto',
               'verbose':1,
               'max_bins' : 255,
               'random_state':636,
              
              }


hist = HistGradientBoostingClassifier(**hist_params)
hist.fit(x_train,y_train)
pred = hist.predict_proba(x_valid)[:, 1]
roc_score = roc_auc_score(y_valid, pred)
print(f"roc_auc_score: {roc_score}")

Binning 0.401 GB of training data: 7.460 s
Binning 0.045 GB of validation data: 0.224 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 31 leaves, max depth = 8, train loss: 0.52718, val loss: 0.52718, in 0.318s
[2/1000] 1 tree, 31 leaves, max depth = 9, train loss: 0.50953, val loss: 0.50950, in 0.306s
[3/1000] 1 tree, 31 leaves, max depth = 8, train loss: 0.49420, val loss: 0.49412, in 0.314s
[4/1000] 1 tree, 31 leaves, max depth = 7, train loss: 0.48071, val loss: 0.48059, in 0.311s
[5/1000] 1 tree, 31 leaves, max depth = 7, train loss: 0.46874, val loss: 0.46856, in 0.313s
[6/1000] 1 tree, 31 leaves, max depth = 9, train loss: 0.45802, val loss: 0.45777, in 0.325s
[7/1000] 1 tree, 31 leaves, max depth = 7, train loss: 0.44837, val loss: 0.44807, in 0.338s
[8/1000] 1 tree, 31 leaves, max depth = 8, train loss: 0.43966, val loss: 0.43930, in 0.326s
[9/1000] 1 tree, 31 leaves, max depth = 8, train loss: 0.43176, val loss: 0.43135, in 0.361s
[10/1000] 1 tree, 31 leaves, max depth = 8

In [90]:
submit = pd.DataFrame()
submit['ID'] = test['ID']
submit['Is_Lead'] = hist.predict_proba(x_test)[:, 1]
submit.head(3)

,ID,Is_Lead
0,VBENBARO,0.033305
1,CCMEWNKY,0.871070
2,VK3KGA9M,0.054181


In [91]:
submit.to_csv("Hist_submit_v3.csv",index=False)